In [680]:
# Initial imports
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
from datetime import datetime
import numpy as np
#import tensorflow as tf

#  Import and read the file.
supply_df = pd.read_csv("C:/Users/Elissa/UCB/dataVizfinal/SupplyData/CAISO-ALL-SUPPLY-DATA2.csv")
supply_df.head()


,Renewables,Natural gas,Large hydro,Imports,Batteries,Nuclear,Coal,Other,Date,Time,DateTime,Unix Timestamp
0,2005.0,8736.0,3428.0,7108.0,-8.0,1616.0,-6.0,0.0,4/10/2018,0:00:00,2018-04-10 00:00:00,1.523318e+09
1,2005.0,8722.0,3425.0,7029.0,1.0,1618.0,-6.0,0.0,4/10/2018,0:05:00,2018-04-10 00:05:00,1.523319e+09
2,2048.0,8739.0,3464.0,6873.0,6.0,1619.0,-6.0,0.0,4/10/2018,0:10:00,2018-04-10 00:10:00,1.523319e+09
3,1979.0,8696.0,3470.0,6744.0,-14.0,1619.0,-6.0,0.0,4/10/2018,0:15:00,2018-04-10 00:15:00,1.523319e+09
4,1980.0,8605.0,3454.0,6757.0,-38.0,1616.0,-6.0,0.0,4/10/2018,0:20:00,2018-04-10 00:20:00,1.523320e+09


In [681]:
supply_df['Sum'] = supply_df['Renewables'] + supply_df['Large hydro'] + supply_df['Imports'] + supply_df['Batteries'] + supply_df['Nuclear'] + supply_df['Coal'] + supply_df['Other'] + supply_df['Natural gas']


In [682]:
supply_df.dtypes

Renewables        float64
Natural gas       float64
Large hydro       float64
Imports           float64
Batteries         float64
Nuclear           float64
Coal              float64
Other             float64
Date               object
Time               object
DateTime           object
Unix Timestamp    float64
Sum               float64
dtype: object

In [683]:
baby_df = supply_df['Time'].str.split(":", expand=True)
print(baby_df)

         0   1   2
0        0  00  00
1        0  05  00
2        0  10  00
3        0  15  00
4        0  20  00
...     ..  ..  ..
284793  23  35  00
284794  23  40  00
284795  23  45  00
284796  23  50  00
284797  23  55  00

[284798 rows x 3 columns]


In [684]:
baby_df[0] = baby_df[0].astype(int)
baby_df[1] = baby_df[1].astype(int)

In [685]:
baby_df= baby_df.drop([2],1)

In [686]:
baby_df.dtypes

0    int32
1    int32
dtype: object

In [687]:
# Rename columns and create years value for calculating demand_EV
baby_df = baby_df.rename(columns={baby_df.columns[0]: 'Hours'})
baby_df = baby_df.rename(columns={baby_df.columns[1]: 'Minutes'})

In [688]:
supply_df['Date'] = pd.to_datetime(supply_df['Date'])

#Create a new column for'Year'
baby_df['Year'] = supply_df['Date'].dt.year
#Convert to integers to be used in computations
baby_df['Year'] = baby_df['Year'].astype(int)



baby_df.head(100)

,Hours,Minutes,Year
0,0,0,2018
1,0,5,2018
2,0,10,2018
3,0,15,2018
4,0,20,2018
...,...,...,...
95,7,55,2018
96,8,0,2018
97,8,5,2018
98,8,10,2018


In [689]:
baby_df['DemandMW'] = 0.0

mask = (baby_df['Year'] == 2018) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)
mask1 = (baby_df['Year'] == 2019) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)
mask2 = (baby_df['Year'] == 2020) & (baby_df['Hours']<=6) | (baby_df['Hours']>=22)


baby_df.loc[mask,'DemandMW'] = 6.2356
baby_df.loc[mask1,'DemandMW'] = 8.2818
baby_df.loc[mask2,'DemandMW'] = 9.9174

In [690]:
supply_df.dtypes

Renewables               float64
Natural gas              float64
Large hydro              float64
Imports                  float64
Batteries                float64
Nuclear                  float64
Coal                     float64
Other                    float64
Date              datetime64[ns]
Time                      object
DateTime                  object
Unix Timestamp           float64
Sum                      float64
dtype: object

In [691]:
baby_df

,Hours,Minutes,Year,DemandMW
0,0,0,2018,6.2356
1,0,5,2018,6.2356
2,0,10,2018,6.2356
3,0,15,2018,6.2356
4,0,20,2018,6.2356
...,...,...,...,...
284793,23,35,2020,9.9174
284794,23,40,2020,9.9174
284795,23,45,2020,9.9174
284796,23,50,2020,9.9174


In [692]:

#Create a date attribute for groupby function 
supply_df['Month-DA'] = supply_df['Date'].dt.strftime('%b')

supply_df['Day-DA'] = supply_df['Date'].dt.day

supply_df['Day-DA'] = supply_df['Day-DA'].astype(str)

def f(row):
    if len(row['Day-DA'])<2:
        val = "0" + row['Day-DA']
    else:
        val = row['Day-DA']
    return: val

supply_df['Day-DA'] = supply_df['Day-DA'].apply(f, axis = 1)

supply_df['Month-DA'] = supply_df['Month-DA'].astype(str)
supply_df['DateAttribute'] = supply_df['Month-DA'] +"-"+ supply_df['Day-DA']


supply_df

TypeError: f() got an unexpected keyword argument 'axis'

In [ ]:
supply_df.dtypes

In [ ]:
def fun(datetimething):
     return datetimething.timetuple().tm_yday

In [ ]:
supply_df['Date'] = supply_df['Date'].apply(fun)
supply_df.head(1300)

In [ ]:
final_df = pd.merge(baby_df, supply_df, left_index=True, right_index=True, how='outer')

In [ ]:
final_df.dtypes

In [ ]:
final_df.iloc[-1]

In [ ]:
df1= pd.DataFrame()

In [ ]:

df1 = final_df.groupby(['DateAttribute','Time'])['Sum'].mean()
df1 = df1.reset_index(level=None)
#df1.index
df1.iloc[:-5]

In [ ]:


df1['3yrAvg(SUM)'] = df1['3yrAvg(SUM)'].round(2)
df1 = df1.sort_values(by=['DateAttribute'])
df1.iloc[-1]

In [ ]:
x = final_df.loc[(final_df['DateAttribute'] == '2-29')]
print(x.index)
y = df1.loc[(final_df['DateAttribute'] == '2-29')]
print(y.index)

In [ ]:
final_df = final_df.drop(final_df.index[197257:197545])
final_df = final_df.reset_index(level=None)

In [ ]:
print(type(final_df))
print(final_df.loc[197256:197560])

In [ ]:
print(len(final_df))
final_df.iloc[-1]

In [ ]:
copy1_df1 = df1.copy()
copy2_df1 = df1.copy()


In [ ]:
new_df1 = df1.append(copy1_df1)

In [ ]:
new_df2 = new_df1.append(copy2_df1)
print(len(new_df2))